# Example to load model from https://doi.org/10.1073/pnas.2120333119

In [1]:
import os
import sys

### path to PYSEQM ###
import torch

### path to PYSEQM ###
sys.path.insert(1, '.../PYSEQM_dev/')

### path to HIPNN ###
sys.path.append('.../hippynn/')


from seqm.seqm_functions.constants import Constants
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin
from seqm.MolecularDynamics import XL_BOMD
from seqm.MolecularDynamics import KSA_XL_BOMD
from hippynn.interfaces.pyseqm_interface.gen_par_full_model import gen_par_full_model

from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### ML part ###
model_file = "PNAS_model.pt"
par_atom_node_name = "SEQM_Atom_Params"
seqm_node_name = "SEQM_Energy"

### This one goes to MD driver and generates HIPNN-SEQM parameters in dynamics
leared_par_generator = gen_par_full_model(model_file, par_atom_node_name, seqm_node_name, device)

from hippynn.experiment.serialization import load_checkpoint
model = torch.load(model_file, map_location=device).to(device)
model.eval()
seqm_parameters0 = model.node_from_name(seqm_node_name).torch_module.energy.seqm_parameters
del model

Determined Inputs: ["Positions(db_name='R')", "Species(db_name='Z_long')"]
Determined Outputs: ['SEQM_Atom_Params.atom_charges']
Determined Targets: []
Device was not specified. Attempting to default to device: cuda:0


## Single point

In [3]:
### create molecule object:
species = torch.as_tensor([[8,6,1,1],
                           [8,6,1,1],
                           [8,8,6,0]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.22,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [1.82,   -0.94,    0.00]
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [1.23,    0.00,    0.00],
                               [1.82,    0.94,    0.00],
                               [0.0,0.0,0.0]            # zero-padding for batching
                              ]
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
    "method": 'PM3', #seqm_parameters0["method"],  # AM1, MNDO, PM#
    "scf_eps": 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
    "scf_converger": [1, 0.0],  # converger used for scf loop
    "sp2": [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
    "elements": seqm_parameters0["elements"],  # [0,1,6,8],
    "learned": seqm_parameters0["learned"],  # learned parameters name list, e.g ['U_ss']
    #"parameter_file_dir": '.../PYSEQM_dev/seqm/params/',  # file directory for other required parameters
    "pair_outer_cutoff": 1.0e10,  # consistent with the unit on coordinates
    'eig' : True,
    'UHF' : False,
}

molecules = Molecule(const, seqm_parameters, coordinates, species, learned_parameters=leared_par_generator).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules,learned_parameters=leared_par_generator)

In [4]:
molecules.Etot

tensor([-441.0159, -441.0159, -694.1156], device='cuda:0')

In [5]:
molecules.force

tensor([[[ 1.0288e+01, -1.1970e-15, -0.0000e+00],
         [-8.0646e+00,  8.1116e-14, -0.0000e+00],
         [-1.1116e+00, -1.5929e+00, -0.0000e+00],
         [-1.1116e+00,  1.5929e+00, -0.0000e+00]],

        [[ 1.0288e+01, -1.1241e-15, -0.0000e+00],
         [-8.0646e+00,  8.1421e-14, -0.0000e+00],
         [-1.1116e+00, -1.5929e+00, -0.0000e+00],
         [-1.1116e+00,  1.5929e+00, -0.0000e+00]],

        [[ 5.9416e+01,  1.0721e+00, -2.8026e-14],
         [-9.1345e+01, -5.4177e+01,  2.5216e-14],
         [ 3.1929e+01,  5.3105e+01,  2.8103e-15],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00]]], device='cuda:0')

## MD

In [6]:
### create molecule object ###

species = torch.as_tensor([[8, 6, 1, 1],
                           [8, 6, 1, 1]], dtype=torch.int64, device=device)

coordinates = torch.tensor(
                            [
                                [[0.014,  0.001,  0.001],
                                 [1.336,  0.001,  0.001],
                                 [1.757,  1.039, -0.001],
                                 [1.757, -1.039,  0.000]],

                                [[0.014,  0.001,  0.001],
                                 [1.336,  0.001,  0.001],
                                 [1.757,  1.039, -0.001],
                                 [1.757, -1.039,  0.000]],
                            ],
                            device=device,
                        )

const = Constants().to(device)
seqm_parameters = {
    "method": 'PM3', # seqm_parameters0["method"],  # AM1, MNDO, PM#
    "scf_eps": 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
    "scf_converger": [0, 0.1],  # converger used for scf loop
                                # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                # [1], adaptive mixing
                                # [2], adaptive mixing, then pulay
    "sp2": [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
    "elements": seqm_parameters0["elements"],  # [0,1,6,8],
    "learned": seqm_parameters0["learned"],  # learned parameters name list, e.g ['U_ss']
    #"parameter_file_dir": '.../PYSEQM_dev/seqm/params/',  # file directory for other required parameters
    "pair_outer_cutoff": 1.0e10,  # consistent with the unit on coordinates
    'eig' : True,
    'UHF' : False,
}

### Pass charges and multiplicity if 'UHF':True. Otherwise, neutral singlet is the default setting
### XL-BOMD supports only closed shell systems at the moment!

# charges = torch.tensor([0,0],dtype=torch.int64, device=device)
# mult = torch.tensor([1,3], device=device)
# molecule = Molecule(const, seqm_parameters, coordinates, species, charges, mult).to(device)

molecule = Molecule(const, seqm_parameters, coordinates, species, learned_parameters=leared_par_generator).to(device)

### BOMD, NVE

In [7]:
dt = 0.1

# create MD object and output files
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'BOMD'}
md0 =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=dt, output=output).to(device)

# initialize velocities with predefined randomization seed.
torch.manual_seed(0)
md0.initialize_velocity(molecule )

# run
_ = md0.run(molecule, 5, remove_com=[True, 1], Info_log=True, learned_parameters=leared_par_generator)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   331.26   1.712752e-01 1.683752e+00 1.855027e+00 ||    412.43   2.132418e-01 1.718149e+00 1.931391e+00 || 
     2   362.83   1.875982e-01 1.664978e+00 1.852576e+00 ||    383.89   1.984875e-01 1.733946e+00 1.932434e+00 || 
     3   397.39   2.054646e-01 1.644547e+00 1.850011e+00 ||    357.96   1.850788e-01 1.748268e+00 1.933347e+00 || 
     4   435.04   2.249344e-01 1.622467e+00 1.847401e+00 ||    334.65   1.730288e-01 1.761127e+00 1.934155e+00 || 
     5   475.95   2.460839e-01 1.598751e+00 1.844835e+00 ||    314.00   1.623530e-01 1.772536e+00 1.934889e+00 || 


### XL-BOMD, NVE

In [8]:
### XL-BOMD supports only closed shell systems at the moment!

species = torch.as_tensor([[8, 6, 1, 1],
                           [8, 6, 1, 1]], dtype=torch.int64, device=device)

coordinates = torch.tensor(
    [
        [[0.014,  0.001,  0.001],
         [1.336,  0.001,  0.001],
         [1.757,  1.039, -0.001],
         [1.757, -1.039,  0.000]],
        
        [[0.014,  0.001,  0.001],
         [1.336,  0.001,  0.001],
         [1.757,  1.039, -0.001],
         [1.757, -1.039,  0.000]],
    ],
    device=device,
)

### Pass charges if 'UHF':True. Otherwise, neutral singlet is the default setting
### XL-BOMD supports only closed shell systems at the moment!

#charges = torch.tensor([0,2],dtype=torch.int64, device=device)
#molecule = Molecule(const, seqm_parameters, coordinates, species, charges).to(device)

molecule = Molecule(const, seqm_parameters, coordinates, species, learned_parameters=leared_par_generator).to(device)

dt = 0.1

xl_bomd_params={'k':6}

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'XL-BOMD'}
md2 =  XL_BOMD(xl_bomd_params=xl_bomd_params,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=dt, output=output).to(device)

torch.manual_seed(0)
md2.initialize_velocity(molecule )
_ = md2.run(molecule, 5, remove_com=[True, 1], Info_log=False, learned_parameters=leared_par_generator)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   331.22   1.712540e-01 1.683773e+00 1.855027e+00 ||    412.37   2.132094e-01 1.718182e+00 1.931391e+00 || 
     2   362.70   1.875308e-01 1.665003e+00 1.852533e+00 ||    383.71   1.983952e-01 1.733976e+00 1.932371e+00 || 
     3   397.19   2.053656e-01 1.644557e+00 1.849923e+00 ||    357.73   1.849597e-01 1.748264e+00 1.933224e+00 || 
     4   434.87   2.248440e-01 1.622490e+00 1.847334e+00 ||    334.48   1.729373e-01 1.761139e+00 1.934076e+00 || 
     5   475.84   2.460288e-01 1.598780e+00 1.844809e+00 ||    313.93   1.623151e-01 1.772553e+00 1.934869e+00 || 
